# Ontology Alignment Experiments: Graph Analysis Pipeline

This notebook demonstrates a complete graph analysis pipeline for ontology alignment:
1. **Exploratory Data Analysis (EDA)** - Understanding the graph structure
2. **Weakly Connected Components (WCC)** - Identifying connected subgraphs
3. **Node Similarity** - Finding similar nodes based on shared properties
4. **Node Embeddings** - Creating vector representations for similarity analysis

In [ ]:
# Setup for Google Colab (skip if running locally)
try:
    import google.colab
    import os
    print("📦 Setting up Colab environment...\n")
    GITHUB_URL = "https://github.com/teutaD/EntityMatching-experimentation.git"

    # Clone repository
    !git clone {GITHUB_URL}

    # Check if clone was successful
    if os.path.exists('EntityMatching-experimentation'):
        os.chdir('EntityMatching-experimentation')
        print("✅ Repository cloned successfully\n")
    else:
        raise Exception("Failed to clone repository. Make sure the URL is correct and the repo is public.")

    # Install dependencies
    print("📦 Installing dependencies...")
    !pip install -q neo4j pandas matplotlib seaborn
    print("✅ Setup complete!")

except ImportError:
    print("✅ Running locally - no setup needed")
except Exception as e:
    print(f"❌ Error during setup: {e}")
    print("\n💡 Solutions:")
    print("   1. Make sure your GitHub repository is PUBLIC")
    print("   2. Or use a personal access token for private repos (see Option 2 above)")
    print("   3. Or manually upload the project files using Colab's file browser")

## Neo4j Database Setup

**Before running this notebook, you need a Neo4j database instance.**

### Option 1: Use Neo4j Sandbox (Recommended for Colab)
1. Go to [Neo4j Sandbox](https://sandbox.neo4j.com/)
2. Create a free account and launch a blank sandbox
3. Click **"Connection details"** and download the credentials file
4. Open the downloaded `.txt` file and copy the values below:
   - **Bolt URL** (e.g., `bolt://54.123.45.67:7687` or `neo4j+s://xxxxx.databases.neo4j.io`)
   - **Username** (usually `neo4j`)
   - **Password** (provided in the file)

### Option 2: Use Your Own Neo4j Instance
If you have your own Neo4j database (local or cloud), use those credentials instead.

⚠️ **Important**: Make sure your Neo4j instance:
- Has the **Graph Data Science (GDS) library** installed
- Is accessible from the internet (if using Colab)
- Has the necessary data loaded

In [ ]:
# 🔐 Neo4j Connection Credentials
# Update these values with your Neo4j Sandbox or instance credentials

NEO4J_URI = "bolt://44.200.0.31"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "gyroscopes-alibis-prints"

# Test connection
print("Testing Neo4j connection...")
try:
    from neo4j import GraphDatabase
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    driver.verify_connectivity()
    driver.close()
    print("✅ Successfully connected to Neo4j!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Verify your credentials are correct")
    print("   2. Check if the Neo4j instance is running")
    print("   3. Ensure port 7687 is accessible (firewall/security groups)")
    print("   4. For Colab: Use Neo4j Sandbox or a cloud-hosted instance")

## Setup and Imports

In [ ]:
!pip install neo4j ydata_profiling

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from neo4j import GraphDatabase
from IPython.display import display, HTML
import json

# Import EDA analyzer - add eda directory to path
eda_path = os.path.join(os.getcwd(), 'eda')
if eda_path not in sys.path:
    sys.path.insert(0, eda_path)
    print(f"✅ Added {eda_path} to Python path")

# Import modules
try:
    from neo4j_analyzer import Neo4jPropertyAnalyzer, PerformanceMonitor
    from neo4j_analyzer.report_generator import ReportGenerator
    print("✅ Successfully imported neo4j_analyzer")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print(f"Current directory: {os.getcwd()}")
    print(f"Directory contents: {os.listdir('.')}")
    if os.path.exists('eda'):
        print(f"EDA directory contents: {os.listdir('eda')}")

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## Configuration

**Analysis Settings:**
(Neo4j credentials were set in the previous cell)

In [ ]:
# Analysis Settings
USE_FAST_MODE = True      # Use Cypher aggregations for large graphs
SAMPLE_SIZE = 50000       # Sample size for standard mode
FETCH_SIZE = 2000         # Batch size for data extraction

print(f"Using Neo4j at: {NEO4J_URI}")
print(f"Fast mode: {USE_FAST_MODE}")

---
# Part 1: Exploratory Data Analysis (EDA)

Understanding the structure and properties of our graph data before running algorithms.

## 1.1 Initialize Analyzer and Explore Database

In [ ]:
# Initialize analyzer
analyzer = Neo4jPropertyAnalyzer(
    uri=NEO4J_URI,
    user=NEO4J_USER,
    password=NEO4J_PASSWORD,
    fetch_size=FETCH_SIZE
)

# Get all node labels in the database
labels = analyzer.get_node_labels()
print(f"Found {len(labels)} node labels in the database:")
for label in labels:
    count = analyzer.get_node_count(label)
    print(f"  - {label}: {count:,} nodes")

## 1.2 Analyze Node Properties

Analyze properties to understand:
- **Categorical properties**: Low cardinality, good for grouping
- **Unique properties**: High cardinality, good for identifiers
- **Property distributions**: Understanding data quality

In [ ]:
# Analyze properties for each label
all_results = {}

for label in labels:
    print(f"\n{'='*60}")
    print(f"Analyzing label: {label}")
    print(f"{'='*60}")

    if USE_FAST_MODE:
        summary = analyzer.get_property_summary_fast(label)
    else:
        summary = analyzer.get_property_summary(label, sample_size=SAMPLE_SIZE)

    all_results[label] = summary
    ReportGenerator.print_summary(summary, label)

## 1.3 Visualize Property Types Distribution

In [ ]:
# Aggregate property types across all labels
property_type_counts = {}

for label, summary in all_results.items():
    for prop_name, prop_info in summary.items():
        prop_type = prop_info.get('type', 'UNKNOWN')
        if prop_type not in property_type_counts:
            property_type_counts[prop_type] = 0
        property_type_counts[prop_type] += 1

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
ax1.pie(property_type_counts.values(), labels=property_type_counts.keys(),
        autopct='%1.1f%%', startangle=90)
ax1.set_title('Property Types Distribution')

# Bar chart
ax2.bar(property_type_counts.keys(), property_type_counts.values())
ax2.set_xlabel('Property Type')
ax2.set_ylabel('Count')
ax2.set_title('Property Types Count')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
# Part 2: Weakly Connected Components (WCC)

**Algorithm**: WCC identifies groups of nodes that are connected to each other, even if the connections are indirect.

**Key Parameters:**
- `GRAPH_NAME`: Name of the GDS graph projection to analyze
- `USER_LABEL`: Node label to focus on (e.g., 'Stream')
- `PROPERTY_LABEL`: Related property nodes (e.g., 'Property')
- `REL_TYPE`: Relationship type connecting nodes (e.g., 'HAS')

**What it does:**
- Finds all connected components in the graph
- Assigns a `component_id` to each node
- Filters components with size > 1 (groups with multiple nodes)
- Writes `component_id_2` property to Stream nodes for downstream analysis

In [ ]:
# WCC Configuration
# Graph projection settings - customize these based on your data
GRAPH_NAME = "stream-user-team-graph"
LABELS = ["Stream", "Team", "Game"]
REL_TYPES = ["VIP", "HAS_TEAM"]

print(f"Graph Projection Configuration:")
print(f"  Graph name: {GRAPH_NAME}")
print(f"  Node labels: {LABELS}")
print(f"  Relationship: {REL_TYPES})")

In [ ]:
# Create Graph Projection (if it doesn't exist)
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver:
    with driver.session() as session:
        # Check if graph already exists
        result = session.run(
            "CALL gds.graph.exists($name) YIELD exists RETURN exists",
            name=GRAPH_NAME
        )
        graph_exists = result.single()["exists"]

        if graph_exists:
            print(f"✅ Graph '{GRAPH_NAME}' already exists")

            # Optionally drop and recreate if you want to update the projection
            # Uncomment the following lines to recreate:
            # print(f"Dropping existing graph...")
            # session.run("CALL gds.graph.drop($name)", name=GRAPH_NAME)
            # graph_exists = False

        if not graph_exists:
            print(f"Creating graph projection '{GRAPH_NAME}'...")
            print(f"  Node labels: {LABELS}")
            print(f"  Relationship: {REL_TYPES})")

            # Create the graph projection using Cypher projection
            session.run(
                "CALL gds.graph.project($name, $nodeLabels, $relConfig)",
                name=GRAPH_NAME,
                nodeLabels=LABELS,
                relConfig=REL_TYPES
            )

            print(f"✅ Graph projection '{GRAPH_NAME}' created successfully")

            # Show graph info
            info = session.run(
                "CALL gds.graph.list($name) "
                "YIELD graphName, nodeCount, relationshipCount "
                "RETURN graphName, nodeCount, relationshipCount",
                name=GRAPH_NAME
            ).single()

            if info:
                print(f"\n📊 Graph Statistics:")
                print(f"  Nodes: {info['nodeCount']:,}")
                print(f"  Relationships: {info['relationshipCount']:,}")

driver.close()

In [ ]:
# Run WCC Algorithm
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver:
    with driver.session() as session:
        # Step 1: Run WCC and write component IDs to nodes
        print("Running WCC algorithm and writing component_id to nodes...")
        session.run(
            "CALL gds.wcc.stream($name) "
            "YIELD nodeId, componentId "
            "WITH gds.util.asNode(nodeId) AS n, componentId "
            "WHERE n:Stream "
            "WITH componentId, collect(n) AS nodes, count(*) AS size "
            "WHERE size > 1 "
            "UNWIND nodes AS n "
            "SET n.component_id = componentId",
            name=GRAPH_NAME,
        )
        print("✅ Component IDs written to nodes")

        # Step 2: Query component statistics (one row per component)
        result = session.run(
            "CALL gds.wcc.stream($name) "
            "YIELD nodeId, componentId "
            "WITH gds.util.asNode(nodeId) AS n, componentId "
            "WHERE n:Stream "
            "WITH componentId, count(*) AS size "
            "WHERE size > 1 "
            "RETURN componentId, size "
            "ORDER BY size DESC, componentId ASC",
            name=GRAPH_NAME,
        )

        # Collect results (one row per component)
        wcc_results = []
        for record in result:
            wcc_results.append({
                'component_id': record['componentId'],
                'size': record['size']
            })

driver.close()

# Display results
wcc_df = pd.DataFrame(wcc_results)
print(f"\nFound {len(wcc_df)} connected components with size > 1")
print(f"Total nodes in components: {wcc_df['size'].sum():,}")
display(wcc_df.head(10))

## 2.1 Visualize Component Size Distribution

In [ ]:
# Visualize component sizes
plt.figure(figsize=(10, 5))

# Histogram of component sizes
plt.hist(wcc_df['size'], bins=50, edgecolor='black')
plt.xlabel('Component Size')
plt.ylabel('Frequency')
plt.title('Distribution of Component Sizes')
plt.yscale('log')

plt.tight_layout()
plt.show()

print(f"\nComponent Statistics:")
print(f"  Mean size: {wcc_df['size'].mean():.2f}")
print(f"  Median size: {wcc_df['size'].median():.2f}")
print(f"  Largest component: {wcc_df['size'].max():,} nodes")
print(f"  Smallest component: {wcc_df['size'].min():,} nodes")

---
# Part 3: Node Similarity

**Algorithm**: Node Similarity finds pairs of nodes that are similar based on their shared neighbors (Jaccard similarity).

**Key Parameters:**
- `GRAPH_NAME`: GDS graph projection (must include component_id property from previous step)
- `NODE_LABELS`: Filter to specific node types (e.g., ['Stream'])
- `THRESHOLD`: Minimum score to consider nodes for string matching
- `similarity > 0`: Only return node pairs with non-zero similarity

**What it does:**
- Compares nodes based on shared properties/neighbors
- Calculates Jaccard similarity: |A ∩ B| / |A ∪ B|
- Returns pairs of similar nodes with similarity scores
- Useful for finding potential duplicates or related entities

In [ ]:
# Node Similarity Configuration
SIMILARITY_GRAPH_NAME = "stream-user-team-graph"
NODE_LABELS = ["Stream"]
THRESHOLD = 0.2

print(f"Running Node Similarity on graph: {SIMILARITY_GRAPH_NAME}")
print(f"Analyzing node labels: {NODE_LABELS}")

In [ ]:
# Run Node Similarity Algorithm (Only Within Same Component)
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver:
    with driver.session() as session:
        print("🔍 Computing similarity only for nodes in the same component...")

        result = session.run(
            "CALL gds.nodeSimilarity.stream($name, { "
            "  nodeLabels: $labels "
            "}) "
            "YIELD node1, node2, similarity "
            "WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity "
            "WHERE similarity > 0 "
            "  AND n1.component_id IS NOT NULL "
            "  AND n2.component_id IS NOT NULL "
            "  AND n1.component_id = n2.component_id "  # ⚠️ Only same component!
            "RETURN n1.id AS node1_id, "
            "       n1.component_id AS node1_component_id, "
            "       n2.id AS node2_id, "
            "       n2.component_id AS node2_component_id, "
            "       similarity "
            "ORDER BY similarity DESC "
            "LIMIT 1000",  # Limit for notebook display
            name=SIMILARITY_GRAPH_NAME,
            labels=NODE_LABELS,
        )

        # Collect results
        similarity_results = []
        for record in result:
            similarity_results.append({
                'node1_id': record['node1_id'],
                'node1_component': record['node1_component_id'],
                'node2_id': record['node2_id'],
                'node2_component': record['node2_component_id'],
                'similarity': record['similarity']
            })

driver.close()

# Display results
similarity_df = pd.DataFrame(similarity_results)
print(f"\n✅ Found {len(similarity_df)} similar node pairs within same components (showing top 1000)")
print(f"   All pairs share the same component_id")
high_score = similarity_df.loc[similarity_df["similarity"] > THRESHOLD]
display(high_score.head(20))

## 3.1 Visualize Similarity Distribution

In [ ]:
# Visualize similarity scores
plt.figure(figsize=(10, 5))

# Histogram of similarity scores
plt.hist(high_score['similarity'], bins=50, edgecolor='black')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.title('Distribution of Similarity Scores (Within Same Component)')

plt.tight_layout()
plt.show()

print(f"\nSimilarity Statistics:")
print(f"  Mean similarity: {high_score['similarity'].mean():.4f}")
print(f"  Median similarity: {high_score['similarity'].median():.4f}")
print(f"  Max similarity: {high_score['similarity'].max():.4f}")
print(f"  Min similarity: {high_score['similarity'].min():.4f}")

## 3.2 Component-Level Analysis

**Note**: Since we computed similarity only for nodes within the same component, let's analyze which components have the most similar node pairs.

In [ ]:
# Analyze similarity by component
if len(similarity_df) > 0:
    component_counts = similarity_df['node1_component'].value_counts().head(10)

    print(f"📊 Component-Level Similarity Analysis:")
    print(f"  Total similar pairs: {len(similarity_df):,}")
    print(f"  Number of components with similar pairs: {similarity_df['node1_component'].nunique()}")
    print(f"\n🔝 Top 10 Components by Number of Similar Pairs:")
    for comp_id, count in component_counts.items():
        comp_data = similarity_df[similarity_df['node1_component'] == comp_id]
        avg_sim = comp_data['similarity'].mean()
        print(f"  Component {comp_id}: {count} pairs (avg similarity: {avg_sim:.4f})")
else:
    print("⚠️  No similar pairs found")

## 3.3 String Similarity on High-Scoring Pairs

For node pairs with high structural similarity, let's also compute **string similarity** on their text properties (e.g., `id`, `description`) to find potential duplicates or closely related entities.

In [ ]:
# String Similarity Configuration
SIMILARITY_THRESHOLD = 0.8  # ⚠️ UPDATE: Only analyze pairs above this threshold
STRING_PROPERTIES = ["id", "description"]  # ⚠️ UPDATE: Properties to compare

print(f"String Similarity Configuration:")
print(f"  Similarity threshold: {SIMILARITY_THRESHOLD}")
print(f"  Properties to compare: {STRING_PROPERTIES}")

In [ ]:
# Filter high-similarity pairs and fetch their properties
from difflib import SequenceMatcher

def string_similarity(str1, str2):
    """Compute string similarity using SequenceMatcher (0.0 to 1.0)"""
    if str1 is None or str2 is None:
        return 0.0
    return SequenceMatcher(None, str(str1), str(str2)).ratio()

# Filter to high-similarity pairs
high_similarity_df = similarity_df[similarity_df['similarity'] >= SIMILARITY_THRESHOLD].copy()

print(f"\n🔍 Analyzing {len(high_similarity_df)} high-similarity pairs (similarity >= {SIMILARITY_THRESHOLD})")

if len(high_similarity_df) > 0:
    # Fetch properties for these nodes
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

    with driver:
        with driver.session() as session:
            # Get unique node IDs
            node_ids = set(high_similarity_df['node1_id'].tolist() + high_similarity_df['node2_id'].tolist())

            print(f"Fetching properties for {len(node_ids)} unique Stream nodes...")

            # Fetch properties
            result = session.run(
                "MATCH (n:Stream) "
                "WHERE n.id IN $ids "
                "RETURN n.id AS id, "
                "       n.description AS description, "
                "       n.total_view_count AS total_view_count, "
                "       n.followers AS followers",
                ids=list(node_ids)
            )

            # Build lookup dictionary
            node_props = {}
            for record in result:
                node_props[record['id']] = {
                    'id': record['id'],
                    'description': record['description'],
                    'total_view_count': record['total_view_count'],
                    'followers': record['followers']
                }

    driver.close()

    # Compute string similarity for each pair
    string_sim_results = []
    for _, row in high_similarity_df.iterrows():
        node1_id = row['node1_id']
        node2_id = row['node2_id']

        if node1_id in node_props and node2_id in node_props:
            props1 = node_props[node1_id]
            props2 = node_props[node2_id]

            # Compute string similarity for each property
            prop_similarities = {}
            for prop in STRING_PROPERTIES:
                if prop in props1 and prop in props2:
                    prop_similarities[f'{prop}_similarity'] = string_similarity(props1[prop], props2[prop])

            string_sim_results.append({
                'node1_id': node1_id,
                'node2_id': node2_id,
                'structural_similarity': row['similarity'],
                'component_id': row['node1_component'],
                **prop_similarities,
                'node1_description': props1.get('description', '')[:50] if props1.get('description') else '',
                'node2_description': props2.get('description', '')[:50] if props2.get('description') else ''
            })

    # Create DataFrame
    string_sim_df = pd.DataFrame(string_sim_results)

    print(f"\n✅ Computed string similarity for {len(string_sim_df)} pairs")
    print(f"\nTop 20 pairs by combined similarity:")

    # Add average string similarity column
    sim_cols = [col for col in string_sim_df.columns if col.endswith('_similarity') and col != 'structural_similarity']
    if sim_cols:
        string_sim_df['avg_string_similarity'] = string_sim_df[sim_cols].mean(axis=1)
        display(string_sim_df.nlargest(20, 'avg_string_similarity'))
    else:
        display(string_sim_df.head(20))
else:
    print(f"\n⚠️  No pairs found with similarity >= {SIMILARITY_THRESHOLD}")


---
# Part 4: Node Embeddings

**Algorithm**: FastRP (Fast Random Projection) creates vector representations of nodes, followed by KNN (K-Nearest Neighbors) to find similar nodes.

**Key Parameters:**
- `GRAPH_NAME`: GDS graph projection name
- `LABELS`: Node labels to include (e.g., ['Stream', 'Game'])
- `REL_TYPES`: Relationship types to consider (e.g., ['MODERATOR', 'VIP', 'CHATTER'])
- `embeddingDimension`: Size of embedding vectors (128)
- `topK`: Number of nearest neighbors to find (10)

**What it does:**
- Creates a graph projection filtered by component_id_2
- Generates 128-dimensional embeddings using FastRP
- Finds top 10 most similar nodes for each node using KNN
- Returns node pairs with similarity scores and component IDs

In [ ]:
# # Node Embeddings Configuration
# EMBEDDING_GRAPH_NAME = "stream-user-team-graph"
# EMBEDDING_LABELS = ["Stream", "Steam"]
# EMBEDDING_DIMENSION = 128
# TOP_K = 10

# print(f"Running Node Embeddings on graph: {EMBEDDING_GRAPH_NAME}")
# print(f"Node labels: {EMBEDDING_LABELS}")
# print(f"Embedding dimension: {EMBEDDING_DIMENSION}")
# print(f"Top K neighbors: {TOP_K}")

In [ ]:
# # Step 1: Create Graph Projection for Embeddings (with component_id_2 filter)
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# with driver:
#     with driver.session() as session:
#         # Check if graph exists
#         result = session.run(
#             "CALL gds.graph.exists($name) YIELD exists RETURN exists",
#             name=EMBEDDING_GRAPH_NAME
#         )
#         graph_exists = result.single()["exists"]

#         if graph_exists:
#             print(f"✅ Graph '{EMBEDDING_GRAPH_NAME}' already exists - skipping creation")
#         else:
#             print(f"📊 Creating graph projection '{EMBEDDING_GRAPH_NAME}' with component_id_2 filter...")

#             # Create Cypher projection filtering nodes with component_id_2
#             session.run(
#                 "CALL gds.graph.project.cypher( "
#                 "  $name, "
#                 "  'MATCH (n) WHERE n.component_id_2 IS NOT NULL AND (n:Stream OR n:Game) RETURN id(n) AS id, labels(n) AS labels', "
#                 "  'MATCH (n)-[r:MODERATOR|VIP|CHATTER]-(m) WHERE n.component_id_2 IS NOT NULL AND m.component_id_2 IS NOT NULL RETURN id(n) AS source, id(m) AS target, type(r) AS type' "
#                 ")",
#                 name=EMBEDDING_GRAPH_NAME
#             )
#             print(f"✅ Graph projection created")

#         # Show graph info
#         info = session.run(
#             "CALL gds.graph.list($name) "
#             "YIELD graphName, nodeCount, relationshipCount "
#             "RETURN graphName, nodeCount, relationshipCount",
#             name=EMBEDDING_GRAPH_NAME
#         ).single()

#         if info:
#             print(f"\n📊 Graph Statistics:")
#             print(f"  Nodes: {info['nodeCount']:,}")
#             print(f"  Relationships: {info['relationshipCount']:,}")

# driver.close()

In [ ]:
# # Step 2: Run FastRP to Generate Embeddings
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# with driver:
#     with driver.session() as session:
#         print(f"🚀 Running FastRP algorithm...")
#         print(f"  Embedding dimension: {EMBEDDING_DIMENSION}")

#         # Run FastRP and mutate the graph with embeddings
#         result = session.run(
#             "CALL gds.fastRP.mutate($name, { "
#             "  embeddingDimension: $dimension, "
#             "  mutateProperty: 'embedding' "
#             "}) "
#             "YIELD nodePropertiesWritten",
#             name=EMBEDDING_GRAPH_NAME,
#             dimension=EMBEDDING_DIMENSION
#         )

#         stats = result.single()
#         print(f"✅ FastRP completed: {stats['nodePropertiesWritten']:,} embeddings created")

# driver.close()

In [ ]:
# Step 3: Run KNN on Embeddings
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# with driver:
#     with driver.session() as session:
#         print(f"🔍 Running KNN algorithm (topK={TOP_K})...")

#         result = session.run(
#             "CALL gds.knn.stream($name, { "
#             "  nodeProperties: ['embedding'], "
#             "  topK: $topK "
#             "}) "
#             "YIELD node1, node2, similarity "
#             "RETURN gds.util.asNode(node1).id AS node1_id, "
#             "       gds.util.asNode(node1).component_id_2 AS component_id_node_1, "
#             "       gds.util.asNode(node2).id AS node2_id, "
#             "       gds.util.asNode(node2).component_id_2 AS component_id_node_2, "
#             "       similarity "
#             "ORDER BY similarity DESC "
#             "LIMIT 1000",
#             name=EMBEDDING_GRAPH_NAME,
#             topK=TOP_K,
#         )

#         # Collect results
#         embedding_results = []
#         for record in result:
#             embedding_results.append({
#                 'node1_id': record['node1_id'],
#                 'component_id_1': record['component_id_node_1'],
#                 'node2_id': record['node2_id'],
#                 'component_id_2': record['component_id_node_2'],
#                 'similarity': record['similarity']
#             })

# driver.close()

# # Display results
# embedding_df = pd.DataFrame(embedding_results)
# print(f"\n✅ Found {len(embedding_df)} embedding-based similar pairs (showing top 1000)")
# display(embedding_df.head(20))

## 4.1 Visualize Embedding-Based Similarity

In [ ]:
# Visualize embedding similarity scores
# plt.figure(figsize=(10, 5))

# # Histogram of embedding similarity scores
# plt.hist(embedding_df['similarity'], bins=50, edgecolor='black', alpha=0.7, label='Embeddings')
# if len(similarity_df) > 0:
#     plt.hist(similarity_df['similarity'], bins=50, edgecolor='black', alpha=0.5, label='Node Similarity')
# plt.xlabel('Similarity Score')
# plt.ylabel('Frequency')
# plt.title('Comparison: Embedding vs Node Similarity')
# plt.legend()

# plt.tight_layout()
# plt.show()

# print(f"\nEmbedding Similarity Statistics:")
# print(f"  Mean similarity: {embedding_df['similarity'].mean():.4f}")
# print(f"  Median similarity: {embedding_df['similarity'].median():.4f}")
# print(f"  Max similarity: {embedding_df['similarity'].max():.4f}")
# print(f"  Min similarity: {embedding_df['similarity'].min():.4f}")

---
# Summary

This notebook demonstrated a complete graph analysis pipeline:

1. **EDA**: Explored the graph structure, node labels, and property distributions
2. **WCC**: Identified connected components and assigned component_id_2 to nodes
3. **Node Similarity**: Found similar nodes based on Jaccard similarity of shared neighbors
4. **Node Embeddings**: Created vector representations and found similar nodes using KNN

## Key Findings:
- Component IDs help group related nodes together
- Node similarity captures structural similarity based on shared connections
- Embeddings capture deeper semantic relationships in the graph
- Both approaches complement each other for comprehensive similarity analysis